<html><font size=6><B>3η εργαστηριακή άσκηση</B><BR>
    <BR><B>Link prediction - Πρόβλεψη συνδέσμων</B></BR>
    </font></html>

<html><font size=4><B>1. Κατασκευή γράφου και υπολογισμός μετρικών</B></font>
    <li><B>1(i).</B> Χρησιμοποιήστε το dataset <a href="http://konect.cc/networks/maayan-pdzbase/"> KONECT (PDZBase) </a>, για να κατασκευάσετε τον μη κατευθυνόμενο (undirected) γράφο $G$. Οι κόμβοι του γράφου αναπαριστούν πρωτεΐνες και οι ακμές την αλληλεπίδραση αυτών. Στην σελίδα <a href="http://konect.cc/networks/maayan-pdzbase/"> KONECT (PDZBase)</a> μπορείτε να βρείτε πληροφορίες για τις ιδιότητες του γράφου.</a><BR>
        <B><U>Υπόδειξη:</U></B> Μπορείτε να ελέγξετε προγραμματιστικά αν το dataset περιέχει διπλότυπα των ακμών με την ίδια ή αντίστροφη σειρά (π.χ. ακμές (a,b) και (b,a)), και να λάβετε υπ' όψιν σας αυτήν σας την παρατήρηση, όπου τυχόν χρειαστεί.<BR></html>

In [1]:
#Libraries
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd

In [ ]:
#Initialze Graph
G = nx.Graph()

#Read the datafrom from out.maayan-pdzbase and check for dubplicate edges
dublicated_edges = []
with open("out.maayan-pdzbase") as f:
    for line in f:
        line = line.strip().split()
        if not G.has_edge(line[0], line[1]):
            G.add_edge(line[0], line[1])
        else:
            dublicated_edges.append(line)


Number of nodes:  212
Number of edges:  244
Number of dublicated edges:  0


 <li><B>1(ii).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G$.</li><BR>

In [ ]:
#Function to calculate all characteristics of the graph:
# Numberr of Nodes, Number of Edges, Mean Degree, Number of Connected Components
def graph_characteristics(G):
    number_of_nodes = G.number_of_nodes()
    number_of_edges = G.number_of_edges()
    mean_degree = np.mean(np.array(list(dict(nx.degree(G)).values())))
    number_of_connected_components = nx.number_connected_components(G)
    return number_of_nodes, number_of_edges, mean_degree, number_of_connected_components

In [7]:
initial_node_count, initial_edge_count, initial_mean_degree, initial_connected_components = graph_characteristics(G)
print("Initial Node Count: ", initial_node_count)
print("Initial Edge Count: ", initial_edge_count)   
print("Initial Mean Degree: {:.4f}".format(initial_mean_degree))
print("Initial Connected Components: ", initial_connected_components)

Initial Node Count:  212
Initial Edge Count:  244
Initial Mean Degree: 2.3019
Initial Connected Components:  19


<li><B>2 Προεργασία για την πρόβλεψη συνδέσμων. 2(i). </B>Υπολογίστε το σύνολο των ασύνδετων ζευγών κόμβων (unconnected pairs) στον γράφο χρησιμοποιώντας τον πίνακα γειτνίασης (adjacency matrix). Δώστε επίσης τον αριθμό τους. Λάβετε υπ' όψιν σας ότι ο γράφος είναι μη κατευθυνόμενος και δεν χρειάζεται να διατρέξετε ολόκληρο τον πίνακα γειτνίασης.</li><BR>

In [ ]:
#Find number of unconnected pairs
unconnected_count = 0
adj_matrix = nx.adjacency_matrix(G).toarray()
n = adj_matrix.shape[0]  # Number of nodes

for i in range(n):
    for j in range(i + 1, n): # Only check the upper triangle of the matrix
        if adj_matrix[i, j] == 0:
            unconnected_count += 1
print(f"Number of unconnected pairs: {unconnected_count}")


Number of unconnected pairs: 22124


<li><B>2(ii).</B> Υπολογίστε το σύνολο των ακμών (removable edges) που μπορούν να αφαιρεθούν από τον γράφο χωρίς να υπάρχει κατάτμηση (splitting) του γράφου (δηλ. ο αριθμός των συνεκτικών συνιστωσών θα πρέπει να παραμένει αμετάβλητος). Υπόδειξη: Ελέγχετε επίσης ότι με αφαίρεση ακμής δεν μειώνεται ο αριθμός των κόμβων (εφόσον δεν υπάρχει ξεχωριστό αρχείο μόνο με κόμβους στο dataset). Δώστε επίσης τον αριθμό των ακμών (removable edges) που μπορούν να αφαιρεθούν υπό αυτές τις προϋποθέσεις.</li><BR>

In [1]:
# removable edges: check splitting and number of nodes
#if ( == initial_connected_components) and ( == initial_node_count): #uncomment and complete (modifications allowed)

<li><B>2(iii).</B> Κατασκευάστε dataframe που συνενώνει τις απαντήσεις των ερωτημάτων 2(i) και 2(ii) (unconnected+removables). Χρησιμοποιήστε την μεταβλητή $link$ που θα παίρνει τιμή 1 ή 0 ανάλογα με την ύπαρξη συνδέσμου ή μη. To dataframe αυτό συγκεντρώνει τα θετικά και αρνητικά δείγματα για την πρόβλεψη των μελλοντικών συνδέσεων (link: target variable). <BR>
    <li><B>2(iv).</B> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 2(ii). <BR>
    <li><B>2(v).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G'$.</li><BR>

In [2]:
#dataframe of removable edges (they will function as positive samples)
#...['link'] = 1  #uncomment and complete

#dataframe of unconnected node pairs (they will function as negative samples)
#...['link'] = 0 # uncomment and complete

#merge in one dataframe

#build graph G'=G_reduced

<html><font size=4><B>3. Πρόβλεψη συνδέσμων με embedding βασισμένο σε τυχαίους περιπάτους (Random Walks) </B></font><BR><BR>
    <li><B>3(i).</B> Εξάγετε τα χαρακτηριστικά (features) του γράφου $G'$, που προέκυψε από την αφαίρεση των ακμών. Για τον σκοπό αυτό χρησιμοποιήστε τον αλγόριθμο <B>Node2vec</B>. Ο αλγόριθμος αντιστοιχίζει τους κόμβους του γράφου μέσω τυχαίων περιπάτων (biased 2nd order random walks) σε ένα χώρο μικρού αριθμού διαστάσεων (low dimensional space) διατηρώντας την μεταξύ τους γειτνίαση.
        <BR>Για την επιλογή των παραμέτρων: 
            <li>p - παράμετρος τυχαίων περιπάτων p που καθορίζει την πιθανότητα “1/p” επιστροφής στον κόμβο προέλευσης (source node), </li>
                <li>q - παράμετρος τυχαίων περιπάτων q που καθορίζει την πιθανότητα  “1/q” μετακίνησης σε κόμβο μακριά από τον κόμβο προέλευσης (source node), </li>
                 <li>dimensions - πλήθος διαστάσεων των Node2Vec embeddings, </li>
                     <li>num_walks - αριθμός περιπάτων από κάθε κόμβο, </li>
                         <li>walk_length - μήκος τυχαίου περιπάτου, </li>
                             <li>window_size - μέγεθος παραθύρου context για τον αλγόριθμο Word2Vec, </li><li>num_iter - αριθμός SGD επαναλήψεων (epochs), </li>
            <li>workers - αριθμός workers για τον Word2Vec,</li>
            χρησιμοποιήστε τις τιμές:   </html>
   
                   

 |   p   |   q   | dimensions|  num_walks  |walk_length |window_size|        workers            |
 |-------|-------|-----------|-------------|------------|-----------|---------------------------|
 |  1.0  |  1.0  |  128      |     10      |    80      |     10    |multiprocessing.cpu_count()|

In [3]:
#install node2vec with conda / pip
#from node2vec import Node2Vec   #uncomment
#import multiprocessing          #uncomment

#generate walks
#node2vec_walker = Node2Vec(, , , , , , ) #uncomment and complete

#train node2vec model
#n2v_model = node2vec_walker.fit(window= , min_count=1) #uncomment and complete

<html><font>
    <li><B>3(ii).</B> Εφαρμόστε τον αλγόριθμο Random Forest Classifier, για να προβλέψετε τις συνδέσεις.</li><BR>
    <li><B>3(iii).</B> Υπολογίστε την ακρίβεια ($accuracy$) του μοντέλου. </li><BR></font></html>

In [4]:
#apply trained Node2Vec model on each node pair of the dataframe "unconnected+removables"
#uncomment:
#x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i,j in zip(df_with_target_var['node_1'], df_with_target_var['node_2'])]

In [5]:
#X_train, X_test, y_train, y_test = train_test_split(, , random_state = 0)  #uncomment and complete

In [6]:
#uncomment whole cell
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score

#classifier = RandomForestClassifier(max_depth=2, random_state=0) 
#classifier.fit()   #uncomment and complete